### Notebook to parse scraped html to produce cleaned text of FCA decisions

Sean Rehaag

License: Creative Commons Attribution-NonCommercial 4.0 International [(CC BY-NC 4.0)](https://creativecommons.org/licenses/by-nc/4.0/). NOTE: Users must also comply with upstream [licensing](https://www.fca-caf.gc.ca/fca-caf_eng/important_eng.html) for the FCA source data, as well as requests on source` urls not to allow indexing of the documents by search engines to protect privacy. As a result, users must not make the data available in formats or locations that can be indexed by search engines.
`
Dataset & Code to be cited as: 

    Sean Rehaag, "Federal Court of Appeal Bulk Decisions Dataset" (2023), online: Refugee Law Laboratory <https://refugeelab.ca/bulk-data/fca/>.

Notes:

(1) Data Source: [Federal Court of Appeal](https://www.fca-caf.gc.ca). 

(2) Unofficial Data: The data are unofficial reproductions of materials on the Federal Court of Appeal website. Links to official versions are included in the dataset.

(3) Non-Affiliation / Endorsement: The data has been collected and reproduced without any affiliation or endorsement from the Federal Court of Appeal

(4) Non-Commerical Use: As indicated in the license, data may be used for non-commercial use (with attribution) only. For commercial use, see the Federal Court of Appeal website's [Terms of Use](https://www.fca-caf.gc.ca/fca-caf_eng/important_eng.html).

(5) Accuracy: Data was collected and processed programmatically for the purposes of academic research. While we make best efforts to ensure accuracy, data gathering of this kind inevitably involves errors. As such the data should be viewed as preliminary information aimed to prompt further research and discussion, rather than as definitive information. 

(6) Limitation: Only includes cases with neutral citation, which began to be used in 2001

(7) Delay: Decisions may take many months to be translated (sometimes over a year). As a result, in the most recent years, decisions may only be available in one language.



In [1]:
# import libraries
from bs4 import BeautifulSoup
import pandas as pd
import re
import pathlib
import json
import random

# set up progress bar
from tqdm import tqdm
tqdm.pandas()

# set paths
in_path = pathlib.Path('d:/scraping/DATA/DECISIONS/FCA/BULK/HTML/')
out_path_raw = pathlib.Path('DATA/fca_raw.jsonl')
out_path_parsed = pathlib.Path('DATA/fca_cases.jsonl')
out_path_parquet = pathlib.Path('DATA/fca_cases.parquet')
out_path_yearly = pathlib.Path('DATA/YEARLY/')

# set years sought
start_year = 2001 
end_year = 2023

# set language sought
language_sought = None  #set to 'en' if english only, set to 'fr' if French only, set to None if all languages


### Load Raw Data

In [2]:
# get list of files (including subdirectories) using pathlib
files = list(in_path.glob('**/*.json'))
print(len(files))

# Load data from files
results = []
for file in tqdm(files):
    with open(file) as f:
        data = json.load(f)
        results.append(data)

# convert list of dictionaries to dataframe
df = pd.DataFrame(results)




16876


100%|██████████| 16876/16876 [00:16<00:00, 999.36it/s] 


In [3]:
# export raw df to jsonl
df.to_json(out_path_raw, orient='records', lines=True)


### Parse Data

In [4]:
# clean dataframe

#remove ?iframe=true from scraped_link
df['scraped_link'] = df['scraped_link'].str.replace('\?iframe=true', '', regex=True)

# remove T and everything after from scraped_timestamp & case_decision_date
df['scraped_timestamp'] = df['scraped_timestamp'].str.replace('T.*', '', regex=True)
df['case_decision_date'] = df['case_decision_date'].str.replace('T.*', '', regex=True)

# remove scraped_status_code, referrer_main_source, referrer_sub_source, referrer_file, referrer_timestamp
df = df.drop(columns=['scraped_status_code', 'referrer_main_source', 'referrer_sub_source', 'referrer_file', 'referrer_timestamp'])

# convert case_year to int and filter for years sought
df['case_year'] = df['case_year'].astype(int)
df = df[df.case_year >= start_year]
df = df[df.case_year <= end_year]

# filter for langauge if desired
if language_sought:
    df = df[df.case_language == language_sought]

# remove cases where no citation (typically orders or errors)
# remove df where citation includes '=' or '-'
df = df[df.case_citation.str.contains('=') == False]
df = df[df.case_citation.str.contains('-') == False]

# if citation2 = citation 1, make citation2 = ''
df['case_citation2'] = df['case_citation2'].where(df['case_citation2'] != df['case_citation'], '')

# change 'scraped_link' column to 'source_url;
df = df.rename(columns={'scraped_link': 'source_url'})

# remove 'case_' from all column names
df.columns = df.columns.str.replace('case_', '')


In [5]:
# Run this in parallel to speed up

import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

# Extract text of cases from html
def get_text(html):

    # if html is None, return None
    if html is None:
        return None
    
    # extract text from class 'entry-content' in page1 html using beautiful soup
    soup = BeautifulSoup(html, 'html.parser')

    # convert <br> to new line to preserve paragraphs
    for br in soup.find_all('br'):
        br.replace_with('\n')

    # Insert newline characters after each <p> tag to preserve paragraphs
    for p in soup.find_all('p'):
        p.insert_after('\n')

    return soup.text

# Convert the pandas DataFrame to a Dask DataFrame
ddf = dd.from_pandas(df, npartitions=16) 

# Apply the function in parallel
ddf['text'] = ddf.html.apply(get_text)

# Convert the Dask DataFrame back to a pandas DataFrame
with dask.config.set(scheduler='processes'):
    with ProgressBar():
        df = ddf.compute()



100%|██████████| 14036/14036 [06:30<00:00, 35.90it/s]


In [6]:
# Clean text of cases
def clean_text(text):

    # remove \xa0
    text = text.replace('\xa0', ' ')

    # Remove multiple whitespaces and preserve paragraphs
    text = '\n'.join([re.sub(r'\s+', ' ', line.strip()) for line in text.split('\n')])
    
    # Remove single newlines
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)

    # Convert multiple newlines to single newlines
    text = re.sub(r'\n+', '\n', text)

    # Remove 'You are being directed to the most recent version...
    if 'You are being directed to the most recent version of the statute which may not be' in text:
        text = text.split('You are being directed to the most recent version of the statute which may not be')[0]

    # Remove 'Vous allez être redirigé vers la version...'
    if '\nVous allez être redirigé vers la version' in text:
        text = text.split('\nVous allez être redirigé vers la version')[0]

    # if '\nDecision Information\n' in text, remove everything before it
    if '\nDecision Information\n' in text:
        text = text.split('\nDecision Information\n')[1]

    # if '\nInformations sur la décision\n' in text, remove everything before it
    if '\nInformations sur la décision\n' in text:
        text = text.split('\nInformations sur la décision\n')[1]

    # Remove all strings '\n[Page #]\n' (with # being a number of up to 4 digits 
    text = re.sub(r'\n\[Page \d{1,3}\]\n', ' ', text)
    
    return text

df['unofficial_text'] = df.text.progress_apply(clean_text)


100%|██████████| 14036/14036 [00:25<00:00, 548.22it/s]


In [7]:
# drop unneeded columns
df = df.drop(columns=['html'])
df = df.drop(columns=['text'])

# rename decision_date to document_date
df = df.rename(columns={'decision_date': 'document_date'})

# add dataset column
df['dataset'] = 'FCA'

# add other column
df['other'] = ''

# reorder columns
list_cols = ['citation',
             'citation2', 
             'dataset', 
             'year', 
             'name', 
             'language', 
             'document_date', 
             'source_url', 
             'scraped_timestamp', 
             'unofficial_text',
             'other']

df = df[list_cols]

# reset index
df = df.reset_index(drop=True)

In [8]:
df.head()

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
0,2001 FCA 1,,FCA,2001,Bouvidard Ltée. c. Commission de l'assurance e...,en,2001-02-01,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2022-08-29,Bouvidard Ltée. c. Commission de l'assurance e...,
1,2001 FCA 10,,FCA,2001,Mclean v. Canada (Minister of Citizenship and ...,en,2001-02-08,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2022-08-29,Mclean v. Canada (Minister of Citizenship and ...,
2,2001 FCA 100,,FCA,2001,Mai v. Canada (Minister of Citizenship and Imm...,en,2001-04-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2022-08-29,Mai v. Canada (Minister of Citizenship and Imm...,
3,2001 FCA 101,,FCA,2001,Mitchell Verification Services Group Inc. v. C...,en,2001-04-03,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2022-08-29,Mitchell Verification Services Group Inc. v. C...,
4,2001 FCA 102,,FCA,2001,Pontbriand v. Canada (Attorney General),en,2001-04-03,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2022-08-29,Pontbriand v. Canada (Attorney General)\nCourt...,


In [9]:
df.tail()

,citation,citation2,dataset,year,name,language,document_date,source_url,scraped_timestamp,unofficial_text,other
14031,2023 CAF 79,,FCA,2023,Telus Communications Inc. c. Fédération canadi...,fr,2023-04-13,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2023-07-04,Telus Communications Inc. c. Fédération canadi...,
14032,2023 CAF 87,,FCA,2023,Lill c. Canada,fr,2023-04-27,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2023-07-04,Lill c. Canada\nBase de données – Cour (s)\nDé...,
14033,2023 CAF 9,,FCA,2023,SUCCESSION DE FEU GAÉTAN GAGNÉ c. SA MAJESTÉ L...,fr,2023-01-18,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2023-04-13,Gagné (Succession) c. Canada\nBase de données ...,
14034,2023 CAF 92,,FCA,2023,Vidéotron Ltée c. Technologies Konek Inc.,fr,2023-05-03,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2023-07-04,Vidéotron Ltée c. Technologies Konek Inc.\nBas...,
14035,2023 CAF 93,,FCA,2023,Association des employeurs maritime c. Syndica...,fr,2023-05-05,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2023-07-04,Association des employeurs maritime c. Syndica...,


### Export data

In [10]:
# export cleaned df to jsonl
df.to_json(out_path_parsed, orient='records', lines=True)

In [11]:
# export cleaned df to parquet
df.to_parquet(out_path_parquet)

In [12]:
# export cleaned df to yearly json files
for year in tqdm(range(start_year, end_year+1)):
    df[df.year == year].to_json(out_path_yearly / f'{year}.json', orient='records', indent=4)

100%|██████████| 23/23 [00:00<00:00, 29.11it/s]
